<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course5/week3/Trigger_word_detection_v1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trigger Word Detection

이 딥 러닝 전문화 과정의 마지막 과제에 오신것을 환영합니다!

이번 주 동영상 강의에서는 음성 인식에 딥 러닝을 적용하는 방법을 배웠습니다. 이 과제에서는 음성 데이터 세트를 구성하고 트리거 단어 감 (키워드 감지 또는 깨우기 단어 감지라고도 함)를 위한 알고리즘을 구현합니다.

* 트리거 단어 감지는 Amazon Alexa, Google Home, Apple Siri 및 Baidu DuerOS와 같은 장치가 특정 단어를 인식하여 깨어날 수 있도록 하는 기술입니다.
* 이번 과제에서 학습시킬 트리거 단어는 "Activate"입니다. "Activate"라는 말을 들을 때마다 차임벨 소리가납니다.
* 이 과제가 끝나면, 직접 녹음한 오디오 클립을 재생할 수 있습니다. 이 때 "Activate"라는 단어를 감지하면 알고리즘이 차임벨을 울리도록 할 수 있습니다.
* 이 과제를 완료 한 후에는 "Activate"라고 말할 때마다 좋아하는 앱을 시작하거나 집의 전등을 켜거나 다른 이벤트를 실행할 수 있도록 확장 할 수도 있습니다.

<img src="arts/sound.png" style="width:1000px;height:150px;">

이 과제에서는 다음을 배우게됩니다.
- 음성 인식 프로젝트 구성
- 오디오 녹음을 합성 및 처리하여 train/dev 데이터 세트 생성
- 트리거 단어 감지 모델 학습 및 예측

이제 시작해보겠습니다. 아래 코드를 실행시켜 필요한 패키지를 불러와 보세요.

In [ ]:
import numpy as np
from pydub import AudioSegment
import random
import sys
import io
import os
import glob
import IPython
from td_utils import *
%matplotlib inline

## 1 - Data synthesis : Creating a speech dataset

트리거 단어 감지 알고리즘을 훈련시키기 위한 데이터셋을 만들어봅시다.
- 음성 데이터셋은 실행하고자 하는 적용 분야 및 환경에 가장 가깝게 만드느 것이 이상적입니다.
- 이 과제의 경우, 일하는 환경(도서관, 집, 사무실, 열린 공간 등등)에서 "Activate"라는 단어를 감지하려고 합니다.
- 따라서 서로 다른 백그라운드 소리에, 긍정 단어("Activate")와 부정 단어("Activate" 이외의 다른 임의의 단어)가 혼합된 음성 데이터셋을 만들어야 합니다.

어떻게 만드는지 확인해봅시다.

### 1.1 - Listening to the data

- 우리의 동료 중 한명이 이 프로젝트를 도복 있으며, 주변 도서관, 카페, 식당, 가정집, 사무실 등등을 방문하여 배경 소음과, 긍정/부정 단어를 말하는 사람들의 음성 스니펫을 녹음했습니다. 이 데이터 세트에는 다양한 억양을 구사하는 사람들의 음성이 포함되어 있습니다.
- `raw_data` 디렉토리에서 긍정/부정 단어 및 배경 소음의 원시 오디오 파일 데이터들을 찾을 수 있습니다. 이 오디오 파일을 사용하여 모델 학습을 위한 데이터 세트를 합성합니다.
  - `activates` 디렉토리에는, 트리거 단어인 "activate" 라는 단어를 말하는 사람들의 음성 데이터가 담겨 있습니다.
  - `negatives` 디렉토리에는 트리거 단어가 아닌 임의의 단어를 말하는 사람들의 음성 데이터가 들어 있습니다.
  - 한 음성 녹음본마다 한 개의 단어가 들어있습니다.
  - `backgrounds` 디렉토리는 서로 다른 환경에서 녹음된 10초간의 배경 소음이 들어 있습니다.

아래 셀을 실행시켜 몇 가지 예시를 직접 들어봅시다.

In [ ]:
IPython.display.Audio("./raw_data/activates/1.wav")

In [ ]:
IPython.display.Audio("./raw_data/negatives/4.wav")

In [ ]:
IPython.display.Audio("./raw_data/backgrounds/1.wav")

위와 같은 세 타입 (positives, negatives, backgrounds) 의 데이터를 통해 라벨링된 데이터셋을 제작합니다. 

## 1.2 - From audio recordings to spectrograms

오디오 녹음이란, 실제로 무엇을 의미하는 것일까요?
- 마이크는 시간에 따른 기압의 변화를 기록합니다. 우리 귀가 소리로 인식하는 것은 사실은 기압의 경미한 변화이기 때문입니다.
- 따라서 오디오 녹음은, 마이크가 감지한 미세한 기압 변화를 측정하는 긴 숫자 목록이라고 생각할 수 있습니다.
- 44100Hz(또는 44100 헤르츠)로 샘플링된 오디오를 사용합니다.
  - 이는 마이크가 초당 44,100개의 숫자를 기록하고 있음을 의미합니다.
  - 따라서 10초 오디오 클립은 441,000개의 숫자로 표시됩니다.($10 \times 44100$)


#### Spectrogram

- 오디오의 원시 데이터로부터, "Activate" 라는 단어를 말했는지의 여부를 파악하는 것은 매우 어렵습니다.
- 시퀸스 모델이 트리거 단어를 감지하는 방법을 보다 쉽게 학습할 수 있도록 오디오의 **Spectrogram**을 계산합니다.
- Spectrogram은 특정 순간에 오디오 클립에 존재하는 서로 다른 주파수의 양을 알려줍니다.
- 신호 처리 혹은 푸리에 변환에 관한 수업을 수강한적이 있는 경우 :
  - Spectrogram은 원시 오디오 데이터 위로 창을 슬라이드하고 푸리에 변환을 사용하여 각 창에서 가장 활성화된 주파수를 계산하는 것으로 구할 수 있습니다.
  - 위 문장을 이해하지 못해도 괜찮습니다. 너무 걱정하지 마세요.

예시를 몇 개 살펴보겠습니다.


In [ ]:
IPython.display.Audio("audio_examples/example_train.wav")

In [ ]:
x = graph_spectrogram("audio_examples/example_train.wav")

위의 그래프는 연속적인 시간(x축)에서 각 주파수가(y축) 얼마나 활성화된 상태인지를 나타냅니다.

<img src="arts/spectrogram.png" style="width:500px;height:200px;">
<center>그림: 오디오 녹음 파일의 Spectrogram</center>

- Spectrogram의 색상은 서로 다른 시점에서 오디오에 서로 다른 주파수가 존재하는 정도를 나타냅니다.
- 녹색은 특정 주파수가 오디오 클립에 더 많이(크게) 활성화되어 있음을 의미합니다.
- 파란색 사각형은 덜 활성화된 주파수를 나타냅니다.
- 출력 spectrogram의 크기는 spectrogram을 만드는 소프트웨어의 하이퍼 파라미터와, 입력의 길이에 따라서 다릅니다.
- 이 과제에서는 훈련 데이터의 표준 길이로 10초의 오디오 클립을 사용합니다.
  - Spectrogram의 time step 수는 5511입니다.
  - 추후 이 spectrogram이 인공 신경망의 입력 $x$가 될 것이므로, $T_x = 5511$이 됩니다.

In [ ]:
_, data = wavfile.read("audio_examples/example_train.wav")
print("Time steps in audio recording before spectrogram", data[:,0].shape)
print("Time steps in input after spectrogram", x.shape)

이제, 여러분은 다음과 같이 정의할 수 있습니다.

In [ ]:
Tx = 5511 # The number of time steps input to the model from the spectrogram
n_freq = 101 # Number of frequencies input to the model at each time step of the spectrogram

#### Dividing into time-intervals

표준 길이인 10초의 간격을 다른 단위로 나눌 수 있습니다.
- 원시 오디오 데이터는 10초를 441,000 단위로 나눕니다.
- Spectrogram은 10초를 5,511 단위로 나눕니다.
  - $T_x = 5511$
- 파이썬 모듈 `pydub`를 사용하여 오디오를 합성하고, 10초를 10,000 단위로 나눕니다.
- 이번 과제에서 학습시킬 모델은, 10초를 1,375 단위로 나눕니다.
  - $T_y = 1375$
  - 1375개의 각 time step에 대해 모델은 누군가가 최근에 트리거 단어인 "Activation"을 말했는지 여부를 예측합니다.
- 위의 모든 값들은 하이퍼 파라미터로, 변경할 수 있습니다(마이크 하드웨어에 종속된 숫자인 441,000 제외)
- 이번 과제에선 음성 시스템에 사용되는 표준 범위 내의 값을 선택했습니다.

In [ ]:
Ty = 1375 # The number of time steps in the output of our model

### 1.3 - Generating a single training example

#### Benefits of synthesizing data
음성 데이터는 데이터 수집과 라벨링이 어렵기 때문에, `activations`, `negatives`, `backgrounds`로 나뉘어진 각각의 데이터를 사용해 훈련 데이터 세트를 합성합니다.
- 임의의 "Activation" 단어가 포함된 10초의 오디오 클립을 매우 많이 녹음하는 것은 상당히 느린 작업입니다.
- 그 대신, 트리거 단어와 부정 단어를 녹음하고 배경 소음을 별도로 녹음(또는 무료 온라인 사이트에서 다운로드)해 이를 합성하는 것이 더 쉽습니다.

#### Process for Synthesizing an audio clip
- 단일 훈련 데이터를 합성하려면 다음의 단계를 수행합니다.
  - 임의의 10초 배경 소음 오디오 클립 선택.
  - 이 10초 클립에 0~4개의 "activate"오디오 클립을 무작위로 삽입합니다.
  - 이 10초 클립에 0~2개의 부정 단어 오디오 클립을 무작위로 삽입합니다.
- 배경 클립에 "activate" 단어를 합성했기 때문에, 10초 클립에서 언제 "activate" 단어가 등장하는지 정확히 알 수 있습니다.
  - 이러면 추후 $y^{\langle t \rangle}$를 라벨링할 때 더욱 쉽게 할 수 있습니다.

#### Pydub
- 오디오 데이터를 조작하려면 `pydub` 패키지를 사용합니다.
- Pydub는 원시 오디오 파일을 pydub 데이터 구조 리스트로 변환합니다.
  - 데이터 구조의 세부 사항을 잘 몰라도 걱정하지 마세요.
- Pydub는 이산 간격으로 1ms를 사용합니다.(1ms = 1/1000초)
  - 이것이 10초 클립이 항상 10,000개의 time step을 사용하여 표현되는 이유입니다.

In [ ]:
# Load audio segments using pydub 
activates, negatives, backgrounds = load_raw_audio()

print("background len should be 10,000, since it is a 10 sec clip\n" + str(len(backgrounds[0])),"\n")
print("activate[0] len may be around 1000, since an `activate` audio clip is usually around 1 second (but varies a lot) \n" + str(len(activates[0])),"\n")
print("activate[1] len: different `activate` clips can have different lengths\n" + str(len(activates[1])),"\n")

### Overlaying positive/negative 'word' audio clips on top of the background audio

- 주어진 10초간의 배경 소음 클립과, 긍정/부정 단어를 포함하고 있는 짧은 오디오 클립에 대하여, 여러분은 배경 소음 위에 이 짧은 단어 오디오 클립을 쌓을 수 있습니다.
- 배경 소음에 긍정/부정 단어의 여러 오디오 클립을 쌓되, 이전에 추가한 다른 클립과 겹치는 어딘가에 단어를 추가하고 싶지는 않습니다.
  - 삽입 시 단어 오디오 클립이 겹치지 않도록, 이전에 삽입한 오디오 클립의 시간을 추적합니다.
- 명확하게 말하자면, 카페에서 녹음된 10초의 배경 소음 클립에 1초의 "Activation"을 삽입하면, **11초 클립으로 끝나지 않습니다.**
  - 합성 결과 오디오 클립은 여전히 10초 길이입니다.
  - 추후 `pydub`를 사용하여 이를 수행하는 방법에 대해 알아봅니다.

#### Label the positive/negative words

- 라벨 데이터 $y^{\langle t \rangle}$는 누군가 "activate"라는 단어를 끝까지 말했는지 여부를 나타냅니다.
- $y^{\langle t \rangle} = 1$는 해당 오디오 클립이 "activate"라고 말한 경우입니다.
- 주어진 배경 소음 오디오 클립에서, 일단 모든 $t$에 대해서 $y^{\langle t \rangle} = 0$으로 초기화 할 수 있습니다. 배경 클립에는 어떤 "activate" 단어도 포함되어 있지 않기 때문입니다.
- "activate" 클립을 삽입하거나 오버레이할 때, $y^{\langle t \rangle}$ 라벨도 업데이트됩니다.
  - 단일 time step의 라벨을 업데이트 하는 대신 대상 라벨 1을 갖추도록 출력의 50 단계를 업데이트합니다.
  - 지난 강의에서 여러 개의 연속된 time step을 업데이트하면 학습 데이터의 균형이 더 잘 맞춰진다는 것을 기억하세요.
- 누군가 "activate"라는 단어를 **끝까지 말했을 때**를 감지하는 GRU(Gated Recurrent Unit)을 학습시킬 것입니다.

#### Example

- 합성된 "activate" 클립이 10초 오디오의 5초쯤 끝났다고 가정합시다(클립의 정확히 중간에)

- $T_y = 1375$이므로, time step $687 = $`int(1375 * 0.5)`은 오디오 클립의 5초 지점에 해당합니다.

- $y^{\langle 688 \rangle} = 1$로 설정합니다.

- 이 시점 이후 짧은 시간 간격동안 GRU가 "activate" 단어를 감지할 수 있도록 하기 위하여, 실제로 **50만큼의 연속적인 값**을 1로 설정합니다.
  - 구체적으로, $ y^{\langle 688 \rangle} = y^{\langle 689 \rangle} = \cdots = y^{\langle 737 \rangle} = 1 $만큼 설정해줍니다.


##### Synthesized data is easier to label
- 이는 훈련 데이터를 합성하는 또 다른 이유입니다. 위에서 설명한 것처럼, $y^{\langle t \rangle}$를 생성하는 것은 비교적 간단합니다.
- 이와 대조적으로, 마이크에 10초 분량의 오디오가 녹음되어 있는 경우 사람이 이를 듣고 "activate" 단어가 끝났을 때 수동으로 라벨을 표시하는 것은 시간이 많이 걸립니다.

#### Visualizing the labels

- 아래는 오디오 클립에서 $y^{\langle t \rangle}$ 라벨을 보여주는 그림입니다.
  - "activate", "innocent", "activate", "baby" 단어를 순차적으로 삽입했습니다.
  - 트리거 단어에 대한 긍정 라벨 1은 해당 트리거 단어를 말했을 때만 연결됩니다.

<img src="arts/label_diagram.png" style="width:500px;height:200px;">
<center>그림 2</center>

#### Helper function

훈련 세트를 합성하는 로직을 구현할 때, 아래의 helper 함수들을 사용할 수 있습니다.

- 이 함수들은 1ms의 이산 간격을 사용합니다.
- 따라서 10초 분량의 오디오 클립은 항상 10,000 단계로 분리됩니다.

1. `get_random_time_segment(segment_ms)` 
  - 배경 소음 오디오에서 임의의 time segment를 검색합니다.
2. `is_overlapping(segment_time, existing_segments`)
  - 해당 time segment가 기존 segment와 겹치는지 확인합니다.
3. `insert_audio_clip(background, audio_clip, existing_times`)
  - 배경 오디오의 임의의 시간에 오디오 클립을 삽입합니다.
  - `get_random_time_segment` 및 `is_overlapping` 함수를 사용하세요.
4. `insert_ones(y, segment_end_ms)`
  - 트리거 단어 "activate" 단어가 종료되는 시점에 벡터 y에 1을 삽입합니다.

#### Get a random time segment
- `get_random_time_segment(segment_ms)` 함수는 `segment_ms` 길이의 오디오 클립을 삽입할 수 있는 임의의 time segment를 반환합니다.
- 아래 코드를 읽고, 이 함수가 뭘 하는지 이해하고 확인하세요.

In [ ]:
def get_random_time_segment(segment_ms):
    """
    Gets a random time segment of duration segment_ms in a 10,000 ms audio clip.
    
    Arguments:
    segment_ms -- the duration of the audio clip in ms ("ms" stands for "milliseconds")
    
    Returns:
    segment_time -- a tuple of (segment_start, segment_end) in ms
    """
    
    segment_start = np.random.randint(low=0, high=10000-segment_ms)   # Make sure segment doesn't run past the 10sec background 
    segment_end = segment_start + segment_ms - 1
    
    return (segment_start, segment_end)

#### Check if audio clips are overlapping

- 배경 소음의 time segment (1000, 1800)과 (3400, 4500)에 오디오 클립을 삽입한다고 가정해봅시다.
  - 첫 번째 segment는 time step 1000에서 시작하고, 1800에서 끝납니다.
  - 두 번째 segment는 time step 3400에서 시작하고, 4500애서 끝납니다.
- 만약 우리가 새로운 오디오 클립을 (3000, 3600) segment에 삽입하고자 한다면, 이 클립이 기존에 이미 삽입된 segment와 겹치는지 아닌지 확인할 필요가 있습니다.
  - 이 경우, (3000, 3600) segment는 기존에 삽입한 (3400, 4500) segment와 겹칩니다. 따라서 우리는 이 segment를 삽입해선 안됩니다.
- 이 함수의 목적을 위해, (100, 200) segment와 (200, 250) segment가 time step 200부터 겹치기 때문에, 해당 결과를 리턴하도록 정의하십시오.
- (100, 199)와 (200, 250)은 겹치지 않습니다.

<br>

**연습 문제**: 주어진 segment가 기존의 segment와 겹치는지 `is_overlapping(segment_time, existing_segments)` 함수를 구현하세요. 두 단계를 거쳐야 합니다.

1. 일단 먼저 "False" 플래그 값을 만들고, 이후 겹치는 영역을 발견했을 경우 "True" 로 변경합니다.
2. previous_segments의 시작~종료 시간에 대해 반복합니다. 이 시간을 segment의 시작-종료 시간과 비교하세요. 겹치는 부분이 있으면 (1)번 단계에 정의된 플래그를 True로 변경합니다.

아래 코드를 확인하세요.
```python
for ....:
        if ... <= ... and ... >= ...:
            ...
```

힌트 : 다음의 경우 두 오디오 세그먼트가 겹친다고 할 수 있습니다.
- 새로운 segment는 이전 segment에 끝나기 전에 시작됩니다.
- **And**
- 새 segment는 이전 segment가 시작된 이후 종료됩니다.

In [ ]:
# GRADED FUNCTION: is_overlapping

def is_overlapping(segment_time, previous_segments):
    """
    Checks if the time of a segment overlaps with the times of existing segments.
    
    Arguments:
    segment_time -- a tuple of (segment_start, segment_end) for the new segment
    previous_segments -- a list of tuples of (segment_start, segment_end) for the existing segments
    
    Returns:
    True if the time segment overlaps with any of the existing segments, False otherwise
    """
    
    segment_start, segment_end = segment_time
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Initialize overlap as a "False" flag. (≈ 1 line)
    overlap = None
    
    # Step 2: loop over the previous_segments start and end times.
    # Compare start/end times and set the flag to True if there is an overlap (≈ 3 lines)
    for previous_start, previous_end in previous_segments:
        if None:
            overlap = None
    ### END CODE HERE ###

    return overlap

In [ ]:
overlap1 = is_overlapping((950, 1430), [(2000, 2550), (260, 949)])
overlap2 = is_overlapping((2305, 2950), [(824, 1532), (1900, 2305), (3424, 3656)])
print("Overlap 1 = ", overlap1)
print("Overlap 2 = ", overlap2)

**모범 답안**:

<table>
    <tr>
        <td>
            **Overlap 1**
        </td>
        <td>
           False
        </td>
    </tr>
    <tr>
        <td>
            **Overlap 2**
        </td>
        <td>
           True
        </td>
    </tr>
</table>

#### Insert audio clip
- 이전에 구현한 helper 함수들을 사용하여, 10초의 배경 소음의 무작위 time step에 audio clip을 삽입하는 함수를 구현해 봅시다.
- 새롭게 추가하는 오디오 segment가 기존에 추가된 다른 audio segment와 겹치지 않는것을 보장해야 합니다.

<br>

**연습 문제**:
- 10초의 배경 소음에 오디오 클립을 삽입하는 `insert_audio_clip()` 함수를 구현하세요.
- 아래 네 가지 단계를 따라야 합니다.
1. 삽입할 오디오 클립의 길이를 구하세요.
  - 삽입할 오디오 클립의 길이와 동일한 길이를 가지는 임의의 time segment를 계산합니다.
2. 해당 time segment가 이전에 삽입한 time segment와 겹치지 않는지 확인하세요.
  - 겹치는 경우 1단계로 돌아가서 새로운 time segment를 선택하세요.
3. 기존 time segment 리스트에 새 time segment를 추가합니다.
  - 따라서, (2)번 단계에서는 삽입한 모든 time segment를 살펴봐야 합니다.
4. pydub를 사용해서 배경 소음 위에 오디오 클립을 쌓아 올립니다. 이 함수는 이미 구현되어 있습니다.

In [ ]:
# GRADED FUNCTION: insert_audio_clip

def insert_audio_clip(background, audio_clip, previous_segments):
    """
    Insert a new audio segment over the background noise at a random time step, ensuring that the 
    audio segment does not overlap with existing segments.
    
    Arguments:
    background -- a 10 second background audio recording.  
    audio_clip -- the audio clip to be inserted/overlaid. 
    previous_segments -- times where audio segments have already been placed
    
    Returns:
    new_background -- the updated background audio
    """
    
    # Get the duration of the audio clip in ms
    segment_ms = len(audio_clip)
    
    ### START CODE HERE ### 
    # Step 1: Use one of the helper functions to pick a random time segment onto which to insert 
    # the new audio clip. (≈ 1 line)
    segment_time = None
    
    # Step 2: Check if the new segment_time overlaps with one of the previous_segments. If so, keep 
    # picking new segment_time at random until it doesn't overlap. (≈ 2 lines)
    while None:
        segment_time = None

    # Step 3: Append the new segment_time to the list of previous_segments (≈ 1 line)
    None
    ### END CODE HERE ###
    
    # Step 4: Superpose audio segment and background
    new_background = background.overlay(audio_clip, position = segment_time[0])
    
    return new_background, segment_time

In [ ]:
np.random.seed(5)
audio_clip, segment_time = insert_audio_clip(backgrounds[0], activates[0], [(3790, 4400)])
audio_clip.export("insert_test.wav", format="wav")
print("Segment Time: ", segment_time)
IPython.display.Audio("insert_test.wav")

**모범 답안**

<table>
    <tr>
        <td>
            **Segment Time**
        </td>
        <td>
           (2254, 3169)
        </td>
    </tr>
</table>

In [ ]:
# Expected audio
IPython.display.Audio("audio_examples/insert_reference.wav")

#### Insert ones for the labels of the positive target

- 바로 직전에 "activate" 오디오 클립을 삽입했다 가정하고, 해당 클립이 끝나는 시점에 $y^{\langle t \rangle}$ 라벨을 업데이트하는 코드를 구현해봅시ㅏㄷ.
- 아래 코드에서 `y`는 $T_y = 1375$이므로, `(1,1375` shape의 벡터입니다.
- "activate" 오디오 클립이 $t$ 시점에 끝나면, $y^{\langle t+1 \rangle} = 1$로 설정하고, 다음 49개의 추가 연속 값을 1로 설정합니다.
  - 대상 단어가 전체 배경 소음 오디오의 끝 부분에 나타나면, 1로 설정해야 하는 추가적인 time step 50개가 없을 수 있습니다.
  - 주어진 벡터에서 유효한 인덱스는 `y[0][0]` 에서 y[0][1374]` 까지이므로, 비록 50개의 추가적인 값을 변경해야 하더라도 `y[0][1375]` 인덱스 이상의 업데이트를 시도하지 마세요. $T_y = 1375$이기 때문입니다.
  - 따라서 "activate"가 $1370$ 단계에서 끝나면, `y[0][1371] = y[0][1372] = y[0][1373] = y[0][1374] = 1` 로만 설정합니다.

<br>

**연습 문제**:
`insert_ones()` 함수를 구현하세요.
- 반복문을 사용해야 합니다.
- 만약 파이썬의 배열 슬라이싱 연산을 사용하면, 반복문 없이도 구현할 수 있습니다.
- 만약 segment가 `segment_end_ms`(10000 step의 이산화 단계)에서 끝난다면,
  - $y$ 출력에 대한 인덱싱으로 변환하려면 (총 $1375$ 단계의 이산화 사용) 다음 공식을 사용합니다.
  ```
  segment_end_y = int(segment_end_ms * Ty/10000.0)
  ```

In [ ]:
# GRADED FUNCTION: insert_ones

def insert_ones(y, segment_end_ms):
    """
    Update the label vector y. The labels of the 50 output steps strictly after the end of the segment 
    should be set to 1. By strictly we mean that the label of segment_end_y should be 0 while, the
    50 following labels should be ones.
    
    
    Arguments:
    y -- numpy array of shape (1, Ty), the labels of the training example
    segment_end_ms -- the end time of the segment in ms
    
    Returns:
    y -- updated labels
    """
    
    # duration of the background (in terms of spectrogram time-steps)
    segment_end_y = int(segment_end_ms * Ty / 10000.0)
    
    # Add 1 to the correct index in the background label (y)
    ### START CODE HERE ### (≈ 3 lines)
    for i in range(None, None):
        if None < None:
            y[0, i] = None
    ### END CODE HERE ###
    
    return y

In [ ]:
arr1 = insert_ones(np.zeros((1, Ty)), 9700)
plt.plot(insert_ones(arr1, 4251)[0,:])
print("sanity checks:", arr1[0][1333], arr1[0][634], arr1[0][635])

**모범 답안**
<table>
    <tr>
        <td>
            **sanity checks**:
        </td>
        <td>
           0.0 1.0 0.0
        </td>
    </tr>
</table>
<img src="images/ones_reference.png" style="width:320;height:240px;">

#### Create a training example

마지막으로, 앞서 구현한 `insert_audio_clip` 함수와 `insert_ones` 함수를 사용하여 새 훈련 데이터 셋을 생성하는 함수를 구현해봅시다.

**연습 문제** : `create_training_example()` 함수를 구현하세요. 아래 단계를 따라서 구현합니다.

1. 라벨 벡터 $y$를 $(1, T_y)$의 shape를 갖는 zero numpy array로 선언합니다.
2. 이미 추가된 segment에 해당하는 빈 리스트를 초기화합니다.
3. 0~4까지의 "activate" 오디오 클립을 무작위로 선택하고, 10초의 배경 소음 오디오 클립에 삽입합니다. 이와 함께 라벨 벡터 $y$의 올바른 포지션에 라벨을 삽입합니다.
4. 0~2까지의 트리거 단어가 아닌 "negative" 오디오 클립을 무작위로 선택하여 10초 클립에 삽입합니다.

In [ ]:
# GRADED FUNCTION: create_training_example

def create_training_example(background, activates, negatives):
    """
    Creates a training example with a given background, activates, and negatives.
    
    Arguments:
    background -- a 10 second background audio recording
    activates -- a list of audio segments of the word "activate"
    negatives -- a list of audio segments of random words that are not "activate"
    
    Returns:
    x -- the spectrogram of the training example
    y -- the label at each time step of the spectrogram
    """
    
    # Set the random seed
    np.random.seed(18)
    
    # Make background quieter
    background = background - 20

    ### START CODE HERE ###
    # Step 1: Initialize y (label vector) of zeros (≈ 1 line)
    y = None

    # Step 2: Initialize segment times as an empty list (≈ 1 line)
    previous_segments = None
    ### END CODE HERE ###
    
    # Select 0-4 random "activate" audio clips from the entire list of "activates" recordings
    number_of_activates = np.random.randint(0, 5)
    random_indices = np.random.randint(len(activates), size=number_of_activates)
    random_activates = [activates[i] for i in random_indices]
    
    ### START CODE HERE ### (≈ 3 lines)
    # Step 3: Loop over randomly selected "activate" clips and insert in background
    for random_activate in random_activates:
        # Insert the audio clip on the background
        background, segment_time = None
        # Retrieve segment_start and segment_end from segment_time
        segment_start, segment_end = None
        # Insert labels in "y"
        y = None
    ### END CODE HERE ###

    # Select 0-2 random negatives audio recordings from the entire list of "negatives" recordings
    number_of_negatives = np.random.randint(0, 3)
    random_indices = np.random.randint(len(negatives), size=number_of_negatives)
    random_negatives = [negatives[i] for i in random_indices]

    ### START CODE HERE ### (≈ 2 lines)
    # Step 4: Loop over randomly selected negative clips and insert in background
    for random_negative in random_negatives:
        # Insert the audio clip on the background 
        background, _ = None
    ### END CODE HERE ###
    
    # Standardize the volume of the audio clip 
    background = match_target_amplitude(background, -20.0)

    # Export new training example 
    file_handle = background.export("train" + ".wav", format="wav")
    print("File (train.wav) was saved in your directory.")
    
    # Get and plot spectrogram of the new recording (background with superposition of positive and negatives)
    x = graph_spectrogram("train.wav")
    
    return x, y

In [ ]:
x, y = create_training_example(backgrounds[0], activates, negatives)

**모범 답안**

<img src="arts/train_reference.png" style="width:320;height:240px;">

이제 생성 한 훈련 데이터를 듣고 위에서 생성한 Spectrogram과 비교할 수 있습니다.


In [ ]:
IPython.display.Audio("train.wav")

**모범 답안**

In [ ]:
IPython.display.Audio("audio_examples/train_reference.wav")

마지막으로 생성 된 단일 훈련 데이터에 대한 관련 라벨을 그래프에 그릴 수 있습니다.

In [ ]:
plt.plot(y[0])

**모범 답안**

<img src="arts/train_label.png" style="width:320;height:240px;">

### 1.4 - Full traning set

- 지금까지 단일 훈련 데이터를 생성하는데 필요한 코드를 구현했습니다.
- 위 프로세스를 사용하여 대규모 훈련 데이터 세트를 생성해보겠습니다.
- 시간을 절약하기 위해, 이번 과제에서는 이미 생성된 훈련 데이터 세트를 제공합니다.

In [ ]:
# Load preprocessed training examples
X = np.load("./XY_train/X.npy")
Y = np.load("./XY_train/Y.npy")

### 1.5 - Development set

- 모델을 테스트하기 위해 25개의 데이터로 구성된 Dev 세트를 녹음했습니다.
- 훈련 데이터가 합성되는 동안 실제 입력과 동일한 분포를 사용하여 dev 세트를 만들고 싶습니다.
- 따라서 사람들이 "activate"라고 말하는 10초짜리 오디오 클립과, 기타 임의의 단어를 녹음하고 수동으로 라벨을 붙였습니다.
- 이는 테스트 세트 배포와 최대한 유사하게 dev 세트를 생성해야 한다는 코스 3 - "Structuring Machine Learning Project"에 설명된 원칙을 따릅니다.
  - 이것이 이번 과제에서의 **Dev set이 합성된 오디오가 아닌 실제 오디오를 사용하는 이유**입니다.

In [ ]:
# Load preprocessed dev set examples
X_dev = np.load("./XY_dev/X_dev.npy")
Y_dev = np.load("./XY_dev/Y_dev.npy")

## 2 - Model

- 데이터셋을 만들었으니, 이제 트리거 단어를 감지하는 모델을 작성하고 훈련해봅시다.
- 모델은 1-D의 컨볼루션 신경망과, GRU 레이어 및 dense 레이어를 사용합니다.
- Keras에 있는 위 레이어들을 불러와 봅시다. 이 과정은 몇 분 정도 소요될 수 있습니다.

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

### 2.1 - Build the model

우리의 목표는 Spectrogram을 입력으로 받아 트리거 단어를 감지했을 때 신호를 출력하는 인공 신경망을 구축하는 것입니다. 이 신경망은 4개의 레이어를 사용합니다.
- 컨볼루션 레이어
- 2개의 GRU 레이어
- Dense 레이어

아래 그림은 이번 인공 신경망에 사용할 아키텍쳐입니다.

<img src="arts/model.png" style="width:600px;height:600px;">
<center>그림 3</center>

#### 1D Convolutional layer
이 모델의 핵심 레이어 중 하나는, 1D 컨볼루션 레이어입니다.(그림 3번의 최하단 영역)

- 5511개의 time step을 가진 spectrogram을 입력합니다. 각 step는 101개의 단위의 벡터를 가지고 있습니다.
- 1375 time step의 출력을 리턴합니다.
- 이 출력은 최종 $T_y = 1375$의 time step을 얻기 위해 여러 레이어에서 추가로 처리됩니다.
- 이 1D 컨볼루션 레이어는 코스 4에서 보았던 2D 컨볼류션 신경망과 유사한 역할을 합니다. 저수준의 특징(feature)를 추출한 다음 더 작은 차원의 출력을 생성할 수 있습니다.
- 연산 비용에 관한 측면에서도, 1D 컨볼루션 레이어는 모델 속도를 높이는데 도움이 됩니다. GRU가 5511개의 time step이 아닌 1375의 time step만 가지고도 처리할 수 있기 때문입니다.

#### GRU, dense and sigmoid

- 두 개의 GRU 레이어는 왼쪽에서 오른쪽으로 흐르는 시퀸스를 입력으로 받습니다.
- Dense 레이어와 시그모이드 함수는 예측 결과인 $y^{\langle t \rangle}$를 생성합니다.
- 라벨 값 $y$가 이진 값(0, 1)이므로, 시그모이드 함수의 출력은 마지막 레이어의 값이 1일 확률을 나타냅니다. 이는 사용자가 "activate"라는 단어를 말했는지 여부를 의미합니다.

#### Undirectional RNN
- 이번 과제에서 우리는 양방향 RNN이 아닌 **단뱡향 RNN**을 사용한다는 것을 잊지 마세요.
- 이는 트리거 단어 감지에서 매우 중요합니다. 트리거 단어를 말한 직후에 감지할 수 있는 기능을 개발해야 하기 때문입니다.
- 양방향 RNN을 사용하는 경우, 오디오 클립의 첫 번째 초에 "activation" 단어를 말했는지 알 수 있으려면 전체 10초의 오디오가 전부 녹음될 때 까지 기다려야 합니다.

#### Implement the model

다음의 네 가지 단계에 따라 모델을 구현해보세요.

**Step 1** : 컨볼루션 레이어. 이 레이어를 구현하기 위해서 `Conv1D()`를 사용하세요. 총 196개의 필터를 사용합니다. 필터 사이즈는 15(`kernel_size=15`)에, stride는 4를 사용합니다. 추가적인 정보는 [conv1d](https://keras.io/layers/convolutional/#conv1d)를 참조하세요.
```Python
output_x = Conv1D(filters=...,kernel_size=...,strides=...)(input_x)
```
- 이 코드를 ReLu 활성화 함수와 함께 사용하세요. 원하는 활성화 함수를 소문자로 된 문자열로 전달할 수 있습니다.
```Python
output_x = Activation("...")(input_x)
```
- 이후 Dropout을 사용합니다. 0.8을 파라미터로 넘겨주세요.
```Python
output_x = Dropout(rate=...)(input_x)
```

**Step 2** : 첫 번째 GRU 레이어입니다. 128개의 unit을 가지는 GRU 레이어를 생성하세요.
```Python
output_x = GRU(units=..., return_sequences = ...)(input_x)
```
- GRU 레이어의 출력으로, 마지막 time step의 예측 대신 전체 시퀸스를 반환하여 모든 GRU의 hidden state가 다음 레이어에 투입될 수 있도록 합니다.
- 위와 마찬가지로 0.8의 rate로 dropout 연산을 수행합니다.
- batch normalization을 수행합니다. 별다른 파라미터를 입력해줄 필요는 없습니다.
```Python
output_x = BatchNormalization()(input_x)
```

**Step 3** : 두 번째 GRU 레이어입니다. 이 값은 첫 번째 GRU와 같은 방법으로 구현하면 됩니다. 
- dropout, batch normalization 이후 한번 더 dropout 연산을 수행하세요.

**Step 4** : time-distributed dense layer를 구현하세요.
```Python
X = TimeDistributed(Dense(1, activation = "sigmoid"))(X)
```
Dense 레이어에서 사용되는 값은 모든 time step에서 동일합니다.

문서 : 
- [Keras documentation on wrappers](https://keras.io/layers/wrappers/).  
- 더 많은 정보는 아래 블로그 문서에서 확인하세요. [How to Use the TimeDistributed Layer in Keras](https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/).

**연습 문제** : 그림 3에 구현된 대로 `model()` 함수를 구현하세요.


In [ ]:
# GRADED FUNCTION: model

def model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
    ### START CODE HERE ###
    
    # Step 1: CONV layer (≈4 lines)
    X = None                                 # CONV1D
    X = None                                 # Batch normalization
    X = None                                 # ReLu activation
    X = None                                 # dropout (use 0.8)

    # Step 2: First GRU Layer (≈4 lines)
    X = None                                 # GRU (use 128 units and return the sequences)
    X = None                                 # dropout (use 0.8)
    X = None                                 # Batch normalization
    
    # Step 3: Second GRU Layer (≈4 lines)
    X = None                                 # GRU (use 128 units and return the sequences)
    X = None                                 # dropout (use 0.8)
    X = None                                 # Batch normalization
    X = None                                 # dropout (use 0.8)
    
    # Step 4: Time-distributed dense layer (see given code in instructions) (≈1 line)
    X = None # time distributed  (sigmoid)

    ### END CODE HERE ###

    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [ ]:
model = model(input_shape = (Tx, n_freq))

모델의 shape를 확인하기 위해 모델 요약을 출력 해 보겠습니다.

In [ ]:
model.summary()

**모범 답안**:

<table>
    <tr>
        <td>
            **Total params**
        </td>
        <td>
           522,561
        </td>
    </tr>
    <tr>
        <td>
            **Trainable params**
        </td>
        <td>
           521,657
        </td>
    </tr>
    <tr>
        <td>
            **Non-trainable params**
        </td>
        <td>
           904
        </td>
    </tr>
</table>

인공 신경망의 출력은 shape (None, 1375, 1)이고, 입력은 (None, 5511, 101) 입니다. 컨볼루션 레이어를 사용해 time step의 수를 5511개에서 1375개로 줄였습니다.

### 2.2 - Fit the model
- 트리거 단어 감지는 훈련하는데 오랜 시간이 걸립니다.
- 시간을 절약하기 위해, 위의 아키텍쳐를 사용한 모델을 사용해 약 3시간동안 GPU로 훈련시킨 모델을 제공합니다. 해당 모델은 약 4000개의 큰 데이터 세트로 훈련되었습니다.
- 모델을 불러와 봅시다.

In [ ]:
model = load_model('./models/tr_model.h5')

아래와 같이 Adam 최적화 함수와 이진 크로스 엔트로피 손실함수를 사용해서 모델을 추가로 훈련할 수 있습니다. 26개의 데이터로 이루어진 작은 훈련 데이터 세트에서 한 epoch에 대해서만 빠르게 훈련합니다.

In [ ]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
model.fit(X, Y, batch_size = 5, epochs=1)

### 2.3 - Test the model

마지막으로, 지금까지 훈련한 모델이 dev 세트에서도 잘 돌아가는지 확인해봅시다.

In [ ]:
loss, acc = model.evaluate(X_dev, Y_dev)
print("Dev set accuracy = ", acc)

꽤 잘 동작하는 것 같습니다.

- 하지만, 정확성이라는 지표는 이 작업에서 그다지 좋은 측정 기준 같지는 않아보입니다.
  - 라벨이 0으로 심하게 치우쳐 있기 때문에, 0만 출력하는 신경망은 정확도가 90%를 약간 넘습니다.
  - F1 점수 또는 정밀도/재현율과 같은 더 유용한 측정 기준을 정의할 수 있습니다.
    - 다만 이번 과제에서는 해당 측정 지표에 대해 신경쓰지 말고, 모델이 일부 예측을 어떻게 수행하는지 경험적으로 확인해보세요.

## 3 - Making Predictions

지금까지 드리거 단어 감지를 위한 모델을 구축했으므로, 이를 사용하여 직접 예측해보겠습니다. 아래 코드 블록은 인공 신경망을 통해 wav 파일에 저장된 오디오에 대한 예측 결과를 리턴합니다.

In [ ]:
def detect_triggerword(filename):
    plt.subplot(2, 1, 1)

    x = graph_spectrogram(filename)
    # the spectrogram outputs (freqs, Tx) and we want (Tx, freqs) to input into the model
    x  = x.swapaxes(0,1)
    x = np.expand_dims(x, axis=0)
    predictions = model.predict(x)
    
    plt.subplot(2, 1, 2)
    plt.plot(predictions[0,:,0])
    plt.ylabel('probability')
    plt.show()
    return predictions

#### Insert a chime to acknowledge the "activate" trigger

- 출력되는 각 time step마다 "activate" 단어를 말했는지에 대한 확률을 예측했다면, 그 확률이 특정 임계치를 초과할 때 차임 벨 사운드를 재생할 수 있습니다.
- $y{\langle t \rangle}$ : "activate"라고 말한 이후의 연속된 50개의 값에 대해 1에 가까울 수 있습니다. 하지만 우리는 단 한번만 차임벨 소리를 내고 싶습니다.
   - 따라서 75개의 출력 time step마다 최대 한 번만 차임 소리를 내도록 삽입합니다.
   - 이렇게 하면 단일 "activate" 이벤트에 대해 두 개의 차임벨이 울리는 것을 방지할 수 있습니다.
   - 이는 컴퓨터 비전에서 non-max suppression이 하는 역할과 유사한 일을 합니다.

In [ ]:
chime_file = "audio_examples/chime.wav"
def chime_on_activate(filename, predictions, threshold):
    audio_clip = AudioSegment.from_wav(filename)
    chime = AudioSegment.from_wav(chime_file)
    Ty = predictions.shape[1]
    # Step 1: Initialize the number of consecutive output steps to 0
    consecutive_timesteps = 0
    # Step 2: Loop over the output steps in the y
    for i in range(Ty):
        # Step 3: Increment consecutive output steps
        consecutive_timesteps += 1
        # Step 4: If prediction is higher than the threshold and more than 75 consecutive output steps have passed
        if predictions[0,i,0] > threshold and consecutive_timesteps > 75:
            # Step 5: Superpose audio and background using pydub
            audio_clip = audio_clip.overlay(chime, position = ((i / Ty) * audio_clip.duration_seconds)*1000)
            # Step 6: Reset consecutive output steps to 0
            consecutive_timesteps = 0
        
    audio_clip.export("chime_output.wav", format='wav')

### 3.3 - Test on dev examples

Dev 세트의 두 오디오 클립에서 모델이 어떻게 작동하는지 살펴보겠습니다. 먼저 두 개의 dev 세트 오디오 클립을 들어보겠습니다.

In [ ]:
IPython.display.Audio("./raw_data/dev/1.wav")

In [ ]:
IPython.display.Audio("./raw_data/dev/2.wav")

이제 위와 같은 오디오 클립에 대해 모델을 실행하고 "activation"을 말한 직후 차임벨 소리가 정상적으로 울리는지 확인합니다!

In [ ]:
filename = "./raw_data/dev/1.wav"
prediction = detect_triggerword(filename)
chime_on_activate(filename, prediction, 0.5)
IPython.display.Audio("./chime_output.wav")

In [ ]:
filename  = "./raw_data/dev/2.wav"
prediction = detect_triggerword(filename)
chime_on_activate(filename, prediction, 0.5)
IPython.display.Audio("./chime_output.wav")

# Congratulations
이 과제를 끝마쳤습니다. 축하합니다!

## Here's what you should remember:

- 데이터 합성은 음성 인식, 특히 트리거 단어 감지에 대한 대규모 훈련 데이터 세트를 만드는 효과적인 방법입니다.
- Spectrogram과 1D 컨볼루션 레이어를 사용하는 것은 오디오 데이터를 RNN, GRU 혹은 LSTM에 전달하기 전에 하는 일반적인 전처리 단계입니다.
- End-to-end 딥러닝 접근 방식을 사용해 매우 효과적인 트리거 단어 감지 시스템을 구축할 수 있습니다

**이것으로 이번 전문화 과정의 모든 과제를 마치신 것을 축하합니다!**

끝까지 저희와 함께 해주시고 딥 러닝을 배우기 위해 노력한 모든 노력에 감사드립니다. 과정을 즐겼기를 바랍니다!

# 4 - Try your own example! (OPTIONAL/UNGRADED)

다음은 이 노트북의 선택 사항이며, 채점이 되지 않는 부분이기 때문에 직접 스스로 오디오 클립을 만들어 모델을 시험해 볼 수 있습니다.

- "activate" 라는 단어와 기타 임의의 단어를 말하는 10초 오디오 클립을 녹음하고, 이를 Coursera 허브에 `myaudio.wav`로 업로드합니다.
- 오디오를 wav 파일로 업로드해야 합니다.
- 오디오가 mp3 등 다른 형식으로 녹음된 경우, 온라인에서 wav로 변환할 수 있는 다양한 방법이 있으니 그 방법을 시도하세요.
- 오디오 녹음이 10초가 아닌 경우 잘 동작하지 않으니, 아래 코드를 사용해 10초로 트리밍 혹은 패딩 하세요.

In [ ]:
# Preprocess the audio to the correct format
def preprocess_audio(filename):
    # Trim or pad audio segment to 10000ms
    padding = AudioSegment.silent(duration=10000)
    segment = AudioSegment.from_wav(filename)[:10000]
    segment = padding.overlay(segment)
    # Set frame rate to 44100
    segment = segment.set_frame_rate(44100)
    # Export as wav
    segment.export(filename, format='wav')


Coursera에 오디오 파일을 업로드 한 후 파일 경로를 아래 변수에 입력하세요.

In [ ]:
your_filename = "audio_examples/my_audio.wav"

In [ ]:
preprocess_audio(your_filename)
IPython.display.Audio(your_filename) # listen to the audio you uploaded 

마지막으로 모델을 사용하여 10 초 오디오 클립에서 activate라고 말하는 시점을 예측하고 차임벨을 울립니다. 경고음이 적절하게 추가되지 않으면 chime_threshold를 조정 해보십시오.


In [ ]:
chime_threshold = 0.5
prediction = detect_triggerword(your_filename)
chime_on_activate(your_filename, prediction, chime_threshold)
IPython.display.Audio("./chime_output.wav")